In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import datetime
from urllib.request import urlopen, Request

In [2]:
r = Request('https://th.investing.com/economic-calendar/', headers={'User-Agent': 'Mozilla/5.0'})
time.sleep(2)
response = urlopen(r).read()
soup = BeautifulSoup(response, "html.parser")
table = soup.find_all(class_ = "js-event-item")
result = []

In [3]:
def event_calendar():
    
    for bl in table:
        event_datetime = bl.get('data-event-datetime', '').strip()
        currency = bl.find(class_="left flagCur noWrap").text.strip()
        intensity_divs = bl.find_all(class_="left textNum sentiment noWrap")
        event = bl.find(class_="left event").text.strip()
        intencity_val = 0
        true_count = 0

        for intence in intensity_divs:
            _true = intence.find_all(class_="grayFullBullishIcon")
            _false = intence.find_all(class_="grayEmptyBullishIcon")

            true_count = len(_true)

            if true_count == 3:
                intencity_val = 3
            elif true_count == 2:
                intencity_val = 2
            else :
                intencity_val = 1
                
        event_datetime = event_datetime.split(' ')
        date=event_datetime[0]
        time=event_datetime[1]
        
        
        result.append({'currency' : currency, 'date' : date, 'time' : time, 'intensity' : intencity_val, 'event':event})

    return result

In [4]:
news = event_calendar()
news_df = pd.DataFrame(news)

In [5]:
display(news_df.shape)
display(news_df.head(10))

(50, 5)

,currency,date,time,intensity,event
0,NZD,2025/06/16,05:30:00,1,ดัชนีชี้วัดผลสำเร็จภาคการบริการ ( พ.ค.)
1,GBP,2025/06/16,06:01:00,1,ดัชนีราคาบ้านจากสถาบัน Rightmove (เดือนต่อเดือน)
2,GBP,2025/06/16,06:01:00,1,ดัชนีราคาบ้านจากสถาบัน Rightmove (ปีต่อปี) (ม...
3,KRW,2025/06/16,06:55:00,1,ปริมาณการส่งออก (ปีต่อปี) ( พ.ค.)
4,KRW,2025/06/16,06:55:00,1,ปริมาณการนำเข้า (ปีต่อปี) ( พ.ค.)
5,KRW,2025/06/16,06:55:00,1,ดุลการค้าของเกาหลีใต้ ( พ.ค.)
6,CNY,2025/06/16,08:30:00,1,ดัชนีราคาบ้านในจีน (ปีต่อปี) ( พ.ค.)
7,CNY,2025/06/16,09:00:00,2,การลงทุนในสินทรัพย์คงที่ของจีน (Fixed Asset In...
8,CNY,2025/06/16,09:00:00,2,ดัชนีการผลิตภาคอุตสาหกรรมของจีน (ปีต่อปี) ( พ...
9,CNY,2025/06/16,09:00:00,2,ผลผลิตอุตสาหกรรมของจีนตั้งแต่ต้นปี (ปีต่อปี) ...


In [6]:
now = datetime.datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M-%S").strip().replace(' ', '_')
news_df.to_csv(f'Data/Event_Calendar/{date_time}.csv', index=False)

In [7]:
print(news_df[news_df['intensity'] == 3][['currency', 'event']])

Empty DataFrame
Columns: [currency, event]
Index: []
